In [1]:
import spacy
from spacy.matcher import Matcher

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
matcher = Matcher(nlp.vocab)
pattern = [
    {'LIKE_EMAIL': True}
]
matcher.add('EMAIL_ADDRESS',[pattern])

In [4]:
doc = nlp('This is my email address: vprinco@gmail.com')

In [7]:
#  this is how we find our matches
matches = matcher(doc)
print(matches)

#the output will look like this
# [(16571425990740197027, 6, 7)] a set of tupples
#  where index 0= 16571425990740197027 is a lexeme
#  and 6 is start index and 7 is end index

[(16571425990740197027, 6, 7)]


In [ ]:
#  now you will see that our label was added to the nlp model with this unique lexime
print(nlp.vocab[matches[0][0]].text)

In [ ]:
with open ('data/wiki_mlk.txt','r') as f:
    text = f.read()

diving deep into matchers

In [ ]:
nlp2 = spacy.load('en_core_web_sm')
matcher = Matcher(nlp2.vocab)
patterns = [{'POS':'PROPN'}]
matcher.add('PROPER_NOUN', [patterns])
doc = nlp2(text)
matches = matcher(doc)

print(len(matches))
for match in matches:
    print(match, doc[match[1]: match[2]])

In [ ]:
#  now to grab multi token pronouns we need to make some twicks to the above code
#  the OP (Operators and quantifiers)
# greedy='LONGEST' which will look for the longest token 
# you will notice greedy='LONGEST' will arrange the tokens from longest to shortest
nlp2 = spacy.load('en_core_web_sm')
matcher = Matcher(nlp2.vocab)
patterns = [{'POS':'PROPN','OP':'+'}]
matcher.add('PROPER_NOUN', [patterns], greedy='LONGEST')
doc = nlp2(text)
matches = matcher(doc)
#  this below line will allow us to sort the tokens to solve the problem of greedy='LONGEST' sorting
matches.sort(key= lambda x: x[1])

print(len(matches))
for match in matches:
    print(match, doc[match[1]: match[2]])

now we can say we want to get those proper nouns but this time the ones that are followed by a verb.


In [ ]:
#  this will be the same code but will will edit the pattern 
nlp2 = spacy.load('en_core_web_sm')
matcher = Matcher(nlp2.vocab)
patterns = [{'POS':'PROPN','OP':'+'}, {'POS':'VERB'}]
matcher.add('PROPER_NOUN', [patterns], greedy='LONGEST')
doc = nlp2(text)
matches = matcher(doc)
matches.sort(key= lambda x: x[1])

print(len(matches))
for match in matches:
    print(match, doc[match[1]: match[2]])

In [25]:
import json
with open ('data/alice.json','r') as f:
    data = json.load(f)
text = data[0][2][0]
print(text)

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'


suppose we are asked to grap all the quotation marks and describe the person speaking

In [27]:
#  first we will clean our text cz in this document there ` instead of '
text = text.replace("`","'")
print(text)

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'


In [38]:
#  now we are going to agrab anything in the quotation marks
#  we'll use the above codes and make some mod

speak_lemmas = ['think', 'say']
nlp2 = spacy.load('en_core_web_sm')
matcher = Matcher(nlp2.vocab)
patterns = [
    {'ORTH':"'"},
    {"IS_ALPHA": True, "OP":"+"},
    {"IS_PUNCT": True, "OP":"*"},
    {"ORTH":"'"},
    {"POS":"VERB", "LEMMA":{"IN":speak_lemmas}},
    {"POS":"PROPN","OP":"+"},
    {'ORTH':"'"},
    {"IS_ALPHA": True, "OP":"+"},
    {"IS_PUNCT": True, "OP":"*"},
    {"ORTH":"'"},
]
matcher.add('PROPER_NOUN', [patterns], greedy='LONGEST')
doc = nlp2(text)
matches = matcher(doc)
matches.sort(key= lambda x: x[1])

print(len(matches))
for match in matches:
    print(match, doc[match[1]: match[2]])

1
(451313080118390996, 47, 67) 'and what is the use of a book,' thought Alice 'without pictures or conversation?'


In [42]:
#  now lets try to add this to owr while book chapters
speak_lemmas = ["think", "say"]
text = data[0][2][0].replace( "`", "'")
matcher = Matcher(nlp.vocab)
pattern1 = [{'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}, {"POS": "VERB", "LEMMA": {"IN": speak_lemmas}}, {"POS": "PROPN", "OP": "+"}, {'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}]
pattern2 = [{'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}, {"POS": "VERB", "LEMMA": {"IN": speak_lemmas}}, {"POS": "PROPN", "OP": "+"}]
pattern3 = [{"POS": "PROPN", "OP": "+"},{"POS": "VERB", "LEMMA": {"IN": speak_lemmas}}, {'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}]
matcher.add("PROPER_NOUNS", [pattern1, pattern2, pattern3], greedy='LONGEST')
for text in data[0][2]:
    text = text.replace("`", "'")
    doc = nlp(text)
    matches = matcher(doc)
    matches.sort(key = lambda x: x[1])
    print (len(matches))
    for match in matches[:10]:
        print (match, doc[match[1]:match[2]])

1
(3232560085755078826, 47, 67) 'and what is the use of a book,' thought Alice 'without pictures or conversation?'
0
0
0
0
0
1
(3232560085755078826, 0, 6) 'Well!' thought Alice
0
0
0
0
0
0
0
1
(3232560085755078826, 57, 68) 'which certainly was not here before,' said Alice
0
0
